In [1]:
import os
from tkinter import filedialog
from tkinter import Tk
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd


In [2]:

def calculate_cagr(end_value, start_value, periods):
    '''
    CAGR stands for "Compound Annual Growth Rate"
    '''
    cagr = (end_value / start_value) ** (1/periods) - 1
    if np.isnan(cagr):
        print(f"Invalid CAGR calculation: end_value={end_value}, start_value={start_value}, periods={periods}")
    return cagr

In [3]:
# Unit tests (sort of) for the calculate_cagr function
assert(calculate_cagr(1,1,1) == 0) 
assert(calculate_cagr(0,1,1) == -1)
assert(calculate_cagr(2,1,1) == 1)
assert(np.round(calculate_cagr(1.331,1,3), decimals=1) == 0.1)

In [4]:
'''
def select_file():
    root = Tk()
    root.withdraw() # we don't want a full GUI, so keep the root window from appearing
    filename = filedialog.askopenfilename() # show an "Open" dialog box and return the path to the selected file
    return filename
'''

In [76]:
# Read in and munge the data
filename = 'GSPC.csv'

def read_and_prepare_data(filename):
    data = pd.read_csv(filename)

    data = data[['Date', 'Close']]
    data.Date = pd.to_datetime(data.Date)

    return data

In [78]:
data = read_and_prepare_data(filename)
data.head(10)

,Date,Close
0,1963-09-25,72.89
1,1963-09-26,72.27
2,1963-09-27,72.13
3,1963-09-30,71.70
4,1963-10-01,72.22
5,1963-10-02,72.30
6,1963-10-03,72.83
7,1963-10-04,72.85
8,1963-10-07,72.70
9,1963-10-08,72.60


In [79]:
max_val = data.Close[-1000:].max()
min_val = data.Close[-1000:].min()
mean_val = data.Close[-1000:].mean()

In [80]:
# TODO: write unit-ish tests for this function
def calculate_exponent(values, fit_param):
    exponent = (np.log(fit_param) - np.log(values[0])) / (len(values) - 1)
    return exponent


In [ ]:
trendline[-1] * (1 + exponent)


In [ ]:

def plot_trend_data(data_file, num_future_years=10, trading_days_per_year=260):
    # Read in the CSV file
    data = pd.read_csv(data_file)
    # Convert the 'Date' column to a datetime object, skipping the header row
    date_col = pd.to_datetime(data['Date'].iloc[1:])
    # Extract the fifth column ("Close")
    col_data = data.iloc[:, 4]
    # Convert the data to a numeric type
    col_data = pd.to_numeric(col_data)

    # Calculate the max, min and average values for the last 1000 data points
    last_series = col_data[-1000:]
    max_val, min_val, average_val = np.max(last_series), np.min(last_series), (np.max(last_series) + np.min(last_series)) / 2

    # Prepare the x-axis for plotting the trendlines with extended future points
    num_future_points = num_future_years * trading_days_per_year
    x_extended = np.arange(len(col_data) + num_future_points)

    # Calculate the trendlines
    trendlines = {}
    cagr = {}  # Stores the Compound Annual Growth Rate for each scenario
    for label, value in [('good', max_val), ('poor', min_val), ('average', average_val)]:
        exponent = (np.log(value) - np.log(col_data.iloc[0])) / (len(col_data) - 1)
        trendline = [col_data.iloc[0]]
        for i in range(1, len(x_extended)):
            trendline.append(trendline[-1] * (1 + exponent))
        trendlines[label] = trendline
        if label == 'average':
            cagr[label] = calculate_cagr(trendline[-1], col_data.iloc[0], len(col_data)/trading_days_per_year + num_future_years)
            #cagr_text = ax.text(len(x_extended), trendline[-1], f"CAGR: {cagr[label]*100:.2f}%", color='blue', fontsize=9, ha='right', va='bottom')

    # Create a new figure and plot the curves
    fig, ax = plt.subplots()
    cagr['average'] = calculate_cagr(trendlines['average'][-1], col_data.iloc[-1], num_future_years)
    cagr_text = ax.text(len(x_extended)-1, trendlines['average'][-1], f"Compounded Growth: {cagr['average']*100:.2f}%", color='blue', fontsize=9, ha='right', va='bottom')

    colors = {'good': 'green', 'poor': 'red', 'average': 'blue'}
    for label, trendline in trendlines.items():
        ax.plot(x_extended, trendline, label=f'Long term trend in {label} case', color=colors[label], alpha=0.3)

    # Extract the filename without extension for use as the plot title
    file_title = os.path.splitext(os.path.basename(data_file))[0]

    # Plot the data
    ax.plot(col_data, linewidth=.3, label=file_title, color='black')

    # Add vertical lines for each year
    for x_val in np.arange(0, len(x_extended), 260):
        ax.axvline(x_val, color='gray', linewidth=0.5, linestyle='--', alpha=0.3)

    # Set x-axis label and tick properties
    ax.set_xlabel('Date')
    ax.set_xticks(np.arange(0, len(x_extended), 260))
    ax.tick_params(axis='x', which='major', labelsize=0.67 * ax.xaxis.get_ticklabels()[0].get_size())

    # Add "click to adjust" text to the blue line
    last_data_x = len(col_data) - 1
    last_data_y = trendlines['average'][last_data_x]
    ax.annotate('click to adjust', xy=(last_data_x, last_data_y), xycoords='data',
            fontsize=9, color='blue', ha='center', va='center', alpha=0.4)

    # Prepare x-axis labels with only the year
    date_ticks = date_col.iloc[np.arange(0, len(col_data), 260)].dt.year.astype(str)
    last_valid_date = date_col.dropna().iloc[-1]
    date_ticks_extended = pd.concat([date_ticks, pd.Series([(last_valid_date + pd.DateOffset(months=i * 12)).strftime('%Y') for i in range(0, num_future_years)])]).reset_index(drop=True)
    ax.set_xticklabels(date_ticks_extended, rotation=45, ha='right')

    # Set y-axis label and plot title
    ax.set_ylabel('Price')
    ax.set_title(file_title)

    # Add a legend to the plot
    ax.legend()

    # Allow moving the average trendline by selecting a point on the plot
    def onclick(event):
        if event.inaxes == ax:
            y_val = event.ydata
            exponent = (np.log(y_val) - np.log(trendlines['average'][0])) / (len(col_data) - 1)
            trendlines['average'] = [col_data.iloc[0]]
            for i in range(1, len(x_extended)):
                trendlines['average'].append(trendlines['average'][-1] * (1 + exponent))
            ax.lines[2].set_ydata(trendlines['average'])
            cagr['average'] = calculate_cagr(trendlines['average'][-1], col_data.iloc[-1], num_future_years)
            cagr_text.set_text(f"Compounded Growth: {cagr['average']*100:.2f}%")
            cagr_text.set_position((len(x_extended), trendlines['average'][-1]))
            fig.canvas.draw()


    # Connect the onclick function to the figure
    fig.canvas.mpl_connect('button_press_event', onclick)

    # Show the plot
    plt.show(block=False)


In [ ]:

# Plot the data from the first file in one window
data_file_path = select_file()
plot_trend_data(data_file_path)

# Keep the plot window open
plt.pause(0.1)
plt.show()
